In [ ]:
%pylab inline
import pylab
try:
    import seaborn as sns  # optional; prettier graphs
except ImportError:
    pass

import numpy as np
import nengo
import nengolib
from nengolib.synapses import PadeDelay

In [ ]:
def plot_error(h, ideal, high=30):
    freqs = np.linspace(0, high, 1000)
    r = h.evaluate(freqs)
    r_ideal = ideal(2.j*np.pi*freqs)
    error = r #- r_ideal
    pylab.figure()
    pylab.plot(freqs, abs(error), label="Power")
    pylab.plot(freqs, error.real, label="Real")
    pylab.plot(freqs, error.imag, label="Imag")
    pylab.show()

c = 0.05 #0.00

h_delay = PadeDelay(3, 4, c)
h_predict = PadeDelay(3, 4, -c) #* nengolib.Lowpass(0.1)
#assert h_predict == (~PadeDelay(4, 3, c))

plot_error(h_delay, lambda s: np.exp(-c*s))
plot_error(h_predict, lambda s: np.exp(c*s), high=10000)



print h_delay.zpk
print h_predict.zpk


In [ ]:
print h_delay.poles, h_delay.zeros
print h_predict.poles, h_predict.zeros

In [ ]:
T = 0.05
dt = 0.001

with nengolib.Network() as model:
    stim = nengo.Node(output=lambda x: np.sin(2*np.pi*x/2)) # nengo.processes.WhiteSignal(T, high=2))
    
    # Build a LinearNetwork that approximations a delay
    subnet = nengolib.networks.LinearNetwork(
        h_predict, n_neurons=200, synapse=0.1,
        radii=1.0, normalizer=nengolib.signal.Controllable(), dt=dt, neuron_type=nengo.neurons.Direct())
    nengo.Connection(stim, subnet.input, synapse=None)

    # Add some probes
    p = nengo.Probe(subnet.output, synapse=0.01)
    p_stim = nengo.Probe(stim, synapse=0.01)
    
sim = nengo.Simulator(model, dt=dt)
sim.run(T)

In [ ]:
pylab.figure()
#pylab.plot(sim.trange(), sim.data[p_stim])
pylab.plot(sim.trange(), sim.data[p])
pylab.show()

In [ ]:
dt = 0.0001
t = np.arange(0, 0.2, dt)
u = np.sin(2*np.pi*t/4)
y = h_delay.filt(u, dt=dt)

pylab.figure()
pylab.plot(t, u)
pylab.plot(t, y)
pylab.show()

In [15]:
import logging; logging.getLogger().setLevel(logging.DEBUG)

import nengo
import numpy as np

for _ in range(2):
    solver = nengo.solvers.LstsqL2(reg=np.random.random())
    solver._hack = np.random.random()
    with nengo.Network(seed=0) as model:
        a = nengo.Ensemble(10, 1)
        nengo.Connection(a, a, solver=solver)
    nengo.Simulator(model)


DEBUG:nengo.builder.network:Network step 1: Building ensembles and nodes
DEBUG:nengo.builder.network:Network step 2: Building subnetworks
DEBUG:nengo.builder.network:Network step 3: Building connections
DEBUG:nengo.cache:Cache miss [36ec2c1f0fb13b2fb43f7889c82b012b77b58835].
DEBUG:nengo.builder.network:Network step 4: Building probes
DEBUG:nengo.builder.network:Network step 1: Building ensembles and nodes
DEBUG:nengo.builder.network:Network step 2: Building subnetworks
DEBUG:nengo.builder.network:Network step 3: Building connections
DEBUG:nengo.cache:Cache miss [3c068fa14e38766e22372f318198a098d05811fe].
DEBUG:nengo.builder.network:Network step 4: Building probes


In [41]:
import numpy as np
import nengo

class CustomSolver(nengo.solvers.LstsqL2):
    def __call__(self, A, Y, rng=None, E=None):
        rng.rand()
        return super(CustomSolver, self).__call__(A, Y, rng=rng, E=E)

nengo.cache.get_default_decoder_cache().invalidate()
    
for _ in range(5):
    rng = np.random.RandomState(seed=0)
    with nengo.Network(seed=0) as model:
        a = nengo.Ensemble(10, 1)
        nengo.Connection(a, a, solver=CustomSolver())
        nengo.Connection(a, a, solver=CustomSolver())
        nengo.Connection(a, a, solver=CustomSolver())
        nengo.Connection(a, a, solver=CustomSolver())

    nengo.Simulator(model)

DEBUG:nengo.builder.network:Network step 1: Building ensembles and nodes
DEBUG:nengo.builder.network:Network step 2: Building subnetworks
DEBUG:nengo.builder.network:Network step 3: Building connections
DEBUG:nengo.cache:Cache miss [e8213cbfe194f8af443598f2b6e4b3ff1720607d].
DEBUG:nengo.cache:Cache miss [cb60e67f4aff4863612005132ac5ebc538089562].
DEBUG:nengo.cache:Cache miss [f00a8a7573e8d5b3ceafafcd4a144e45c4ca4b8a].
DEBUG:nengo.cache:Cache miss [fc4ff6a5e5bd2056d91a5a26343975141213f92b].
DEBUG:nengo.builder.network:Network step 4: Building probes
DEBUG:nengo.builder.network:Network step 1: Building ensembles and nodes
DEBUG:nengo.builder.network:Network step 2: Building subnetworks
DEBUG:nengo.builder.network:Network step 3: Building connections
DEBUG:nengo.cache:Cache hit [e8213cbfe194f8af443598f2b6e4b3ff1720607d]: Loaded stored decoders.
DEBUG:nengo.cache:Cache hit [cb60e67f4aff4863612005132ac5ebc538089562]: Loaded stored decoders.
DEBUG:nengo.cache:Cache hit [f00a8a7573e8d5b3ceafa

In [32]:
from nengo.solvers import LstsqL2

solver1 = LstsqL2()
solver2 = LstsqL2()

solver1.foo = lambda bar: None
solver2.foo = lambda bar: None

print nengo.cache.Fingerprint(solver1)
print nengo.cache.Fingerprint(solver2)

FingerprintError: Can't pickle <type 'function'>: attribute lookup __builtin__.function failed

In [ ]:
class